In [14]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import re
import string

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Jonathan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Jonathan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Jonathan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Jonathan\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [15]:
train_set = pd.read_csv("C:/Users/Jonathan/Downloads/south-african-language-identification-hack-2023/train_set.csv")
test_set = pd.read_csv("C:/Users/Jonathan/Downloads/south-african-language-identification-hack-2023/test_set.csv")

In [16]:
def preprocess_text(text):
    
    text = text.lower()
   
    text = text.translate(str.maketrans('', '', string.punctuation))
  
    text = re.sub(r'\d+', '', text)
 
    tokens = word_tokenize(text)
   
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    lemmatizer = WordNetLemmatizer()
    lemmatized_output = ' '.join([lemmatizer.lemmatize(w) for w in tokens])
    
    return lemmatized_output


train_set['cleaned_text'] = train_set['text'].apply(preprocess_text)
test_set['cleaned_text'] = test_set['text'].apply(preprocess_text)

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()

X = tfidf.fit_transform(train_set['cleaned_text'])
y = train_set['lang_id']


In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

lr_model = LogisticRegression(max_iter=1000)
lr_model.fit(X_train, y_train)

cv_scores = cross_val_score(lr_model, X, y, cv=5)
cv_mean = cv_scores.mean()
cv_std = cv_scores.std()

In [19]:
test_set_features = tfidf.transform(test_set['cleaned_text'])

predictions = lr_model.predict(test_set_features)

In [21]:
submission = pd.DataFrame({
    'index': test_set['index'],
    'lang_id': predictions
})

submission.to_csv('submission3.csv', index=False)
